In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Pancreatic_Cancer"
cohort = "GSE120127"

# Input paths
in_trait_dir = "../../input/GEO/Pancreatic_Cancer"
in_cohort_dir = "../../input/GEO/Pancreatic_Cancer/GSE120127"

# Output paths
out_data_file = "../../output/preprocess/Pancreatic_Cancer/GSE120127.csv"
out_gene_data_file = "../../output/preprocess/Pancreatic_Cancer/gene_data/GSE120127.csv"
out_clinical_data_file = "../../output/preprocess/Pancreatic_Cancer/clinical_data/GSE120127.csv"
json_path = "../../output/preprocess/Pancreatic_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Role of BAP1 in pancreatic cancer"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell line: pancreatic cancer cell line PANC-1'], 1: ['genotype: Wildtype', 'genotype: BAP1 deletion (sgBAP1)'], 2: ['treatment: Control', 'treatment: IR (10Gy)']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background info, this appears to be a study of pancreatic cancer cell lines
# with gene expression data comparing wild type and knockout conditions
is_gene_available = True

# 2.1 Data Availability
# Looking at the sample characteristics:
# Row 0: Sex data is available
# Row 1: Contains cell line info, not variable between samples
# Row 2: Contains genotype info that indicates pancreatic cancer samples with different genotypes

# For trait (Pancreatic_Cancer):
# All samples are pancreatic cancer cell lines, so there is no control vs. disease distinction
# Row 2 contains genotype information which we can use to differentiate samples by BAP1 status
trait_row = 2

# For age:
# No age information is available in the sample characteristics
age_row = None

# For gender:
# Row 0 contains sex information
gender_row = 0

# 2.2 Data Type Conversion
def convert_trait(value):
    """
    Convert genotype information to binary values:
    1 for BAP1 knockout (KO) samples
    0 for BAP1 wild-type (WT) samples
    """
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert based on BAP1 status
    if "Bap1 KO" in value:
        return 1  # BAP1 knockout
    elif "Bap1 WT" in value:
        return 0  # BAP1 wild-type
    else:
        return None

def convert_gender(value):
    """
    Convert gender information to binary values:
    0 for female
    1 for male
    """
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert based on sex
    if value.upper() in ["F", "FEMALE"]:
        return 0
    elif value.upper() in ["M", "MALE"]:
        return 1
    else:
        return None

# Age conversion function (not used but defined for completeness)
def convert_age(value):
    """Placeholder function as age data is not available"""
    return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Initial validation and saving of cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We only proceed if trait_row is not None
if trait_row is not None:
    # Extract clinical features using the library function
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save the clinical data to the specified output file
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM3392980': [nan, nan], 'GSM3392981': [nan, nan], 'GSM3392982': [nan, nan], 'GSM3392983': [nan, nan], 'GSM3392984': [nan, nan], 'GSM3392985': [nan, nan], 'GSM3392986': [nan, nan], 'GSM3392987': [nan, nan]}
Clinical data saved to ../../output/preprocess/Pancreatic_Cancer/clinical_data/GSE120127.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be numerical IDs (starting with "17200...") rather than standard human gene symbols
# Human gene symbols typically look like "BRCA1", "TP53", "KRAS", etc.
# These are likely probe IDs or some other platform-specific identifiers that need mapping to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the gene identifiers in the gene expression data (numerical IDs starting with "17200..."),
# and the gene annotation dataframe, we need to find the matching columns.
# Looking at the annotation dataframe, the 'ID' column seems to contain probe IDs,
# and 'Gene Symbol' column contains the gene symbols we need.

# 2. Create a gene mapping dataframe using these two columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 3. Apply gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print information about the resulting gene expression dataframe
print(f"Mapped gene expression data: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("First few gene symbols:")
print(gene_data.index[:5])


Mapped gene expression data: 19963 genes × 8 samples
First few gene symbols:
Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since we determined in Step 2 that trait_row is None (no clinical variation)
# and is_trait_available is False, we can't proceed with linking clinical and gene data
# Instead, we need to create dummy clinical data with a constant trait value

# Create minimal clinical dataframe with samples matching gene data
sample_ids = normalized_gene_data.columns
dummy_clinical_df = pd.DataFrame(index=['Pancreatic_Cancer'], 
                                  columns=sample_ids,
                                  data=[[1] * len(sample_ids)])  # All samples are pancreatic cancer

print(f"Created dummy clinical data with shape: {dummy_clinical_df.shape}")

# 2. Link the clinical and genetic data (even though clinical data is just a constant)
linked_data = geo_link_clinical_genetic_data(dummy_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# Identify trait column
trait_col = linked_data.columns[0]
print(f"Using trait column: {trait_col}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col)
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
# Since we know all samples are pancreatic cancer, this trait is severely biased
is_trait_biased = True
print("The trait is severely biased (all samples are pancreatic cancer).")
unbiased_linked_data = linked_data

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=False,  # We previously determined trait data is not available (no variation)
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data for pancreatic cancer cell lines, but lacks normal controls."
)

# 6. Since the data is not usable (no trait variation), don't save it
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data quality check failed. Dataset lacks necessary trait variation for association studies.")

Normalized gene data saved to ../../output/preprocess/Pancreatic_Cancer/gene_data/GSE120127.csv
Created dummy clinical data with shape: (1, 8)
Linked data shape: (8, 19759)
Using trait column: Pancreatic_Cancer


Shape after handling missing values: (8, 19759)
The trait is severely biased (all samples are pancreatic cancer).
A new JSON file was created at: ../../output/preprocess/Pancreatic_Cancer/cohort_info.json
Data quality check failed. Dataset lacks necessary trait variation for association studies.
